In [1]:
#pip install dash

In [7]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("G:/programming/IBM/IBM - Google Data Analytics Professional Certificate/questions/10 Applied Data Science Capstone/Done/3/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create dropdown options
launch_sites = [{'label': 'All Sites', 'value': 'All Sites'}]
for site in spacex_df["Launch Site"].unique():
    launch_sites.append({'label': site, 'value': site})

# App layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    dcc.Dropdown(id='site-dropdown',
                 options=launch_sites,
                 value='All Sites',
                 placeholder="Select a Launch Site here",
                 searchable=True),
    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    dcc.RangeSlider(id='payload-slider',
                    min=0,
                    max=10000,
                    step=1000,
                    value=[min_payload, max_payload],
                    marks={2500: '2500', 5000: '5000', 7500: '7500'}),

    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2: Pie chart callback
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def select(inputt):
    if inputt == 'All Sites':
        new_df = spacex_df.groupby(['Launch Site'])["class"].sum().reset_index()
        fig = px.pie(new_df, values='class', names='Launch Site',
                     title='Total Success Launches by Site')
    else:
        site_df = spacex_df[spacex_df["Launch Site"] == inputt]
        counts = site_df['class'].value_counts().reset_index()
        counts.columns = ['class_result', 'count']
        counts['name'] = counts['class_result'].map({1: 'Success', 0: 'Failure'})
        fig = px.pie(counts, values='count', names='name',
                     title=f'Total Success vs. Failure for {inputt}')
    return fig

# TASK 4: Scatter chart callback
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)
def scatter(input1, input2):
    low, high = input2
    if input1 == 'All Sites':
        filtered_df = spacex_df[(spacex_df["Payload Mass (kg)"] >= low) &
                                (spacex_df["Payload Mass (kg)"] <= high)]
    else:
        filtered_df = spacex_df[(spacex_df["Launch Site"] == input1) &
                                (spacex_df["Payload Mass (kg)"] >= low) &
                                (spacex_df["Payload Mass (kg)"] <= high)]
    
    fig2 = px.scatter(filtered_df, x="Payload Mass (kg)", y="class",
                      color="Booster Version Category",
                      title='Correlation between Payload and Success')
    return fig2

# Run the app
if __name__ == '__main__':
    app.run(port=8080)
